In [1]:
from project_heart.lv import LV
import numpy as np
import pyvista as pv
pv.set_jupyter_backend("pythreejs")

import os
from pathlib import Path

from project_heart.enums import *

In [2]:
filepath = Path("../sample_files/lvtetmesh_coarse_fibers_LDRB_1.vtk") 
lv = LV.from_pyvista_read(filepath)

In [3]:
lv.create_nodesets_from_regions(mesh_data=LV_MESH_DATA.APEX_BASE_REGIONS, overwrite=False)
lv.create_nodesets_from_regions(mesh_data=LV_MESH_DATA.EPI_ENDO, overwrite=False)
lv.create_nodesets_from_regions(mesh_data=LV_MESH_DATA.SURFS, overwrite=False)
lv.create_nodesets_from_regions(mesh_data=LV_MESH_DATA.AM_SURFS, overwrite=False)
lv.create_nodesets_from_regions(mesh_data=LV_MESH_DATA.SURFS_DETAILED, overwrite=False)

In [4]:
s = lv.create_surface_oi_from_surface(LV_MESH_DATA.SURFS_DETAILED)

In [5]:
lv.set_aortic_info()
lv.set_mitral_info()

In [6]:
aortic_bc = lv.create_spring_rim_bc("AORTIC_BC", LV_SURFS.BORDER_AORTIC)
mitral_bc = lv.create_spring_rim_bc("MITRAL_BC", LV_SURFS.BORDER_MITRAL)

In [7]:
# lv.to_json("sample_lvhex_wfibers.json", 
#            nodeset_enums=[LV_SURFS],
#            surfaces_oi_enums=[LV_SURFS],
#            mesh_cell_data=[LV_FIBERS.F0.value, LV_FIBERS.S0.value, LV_FIBERS.N0.value])

In [8]:
N_NODES = lv.mesh.n_points
N_NODES_AORTIC = len(lv.get_bc("AORTIC_BC")[1]["RIM_NODES"])

In [9]:
adjusted_discrete_sets = {}
to_adj = N_NODES
for key, values in lv._discrete_sets.items():
  adj_vals = np.copy(values) + 1
  adj_vals[:, 1] += to_adj
  adjusted_discrete_sets[key] = adj_vals
  to_adj += len(lv.get_bc(key)[1]["RIM_NODES"]) # account for nodes

In [10]:
from febio_python.feb import FEBio_feb
# ----------------------
# Read template file
feb_template_path = Path("../sample_files/sample_lv_template.feb")
feb = FEBio_feb.from_file(feb_template_path)
feb

FEBio_feb:
-> Module: 0
-> Control: 19
-> Material: 2
--> material 'Holzapfel_Ogden_PAQ': 13
--> material 'rigid body': 2
-> Globals: 1
-> Boundary: 1
-> Loads: 1
-> Discrete: 2
-> LoadData: 3
-> Output: 1
-> MeshData: 0

In [11]:
# ----------------------
# Add Nodes
feb.add_nodes([
  {"name": "LV", 
   "nodes": np.round(lv.nodes(), 4),
   },
  {"name": "AORTIC_RIM", 
   "nodes": np.round(lv.get_bc("AORTIC_BC")[1]["RIM_NODES"], 4)
   },
  {"name": "MITRAL_RIM", 
   "nodes": np.round(lv.get_bc("MITRAL_BC")[1]["RIM_NODES"], 4)
   },
  ])

# ----------------------
# Add Elements
feb.add_elements([
  {
    "name": "LV", 
    "type": "tet4",
    "mat": "1",
    "elems": lv.cells(VTK_ELEMENTS.TETRA) + 1
    },
  {
    "name": "AORTIC_RIM_ELEMS", 
    "type": "quad4",
    "mat": "2",
    "elems": aortic_bc[LV_RIM.ELEMENTS.value] + N_NODES + 1 # adjust element ids
    },
  {
    "name": "MITRAL_RIM_ELEMS", 
    "type": "quad4",
    "mat": "2",
    "elems": mitral_bc[LV_RIM.ELEMENTS.value] + N_NODES + N_NODES_AORTIC + 1 # adjust element ids
    }
  ])

# ----------------------
# Add Nodesets
feb.add_nodesets(lv.get_nodesets_from_enum(LV_SURFS))

# ----------------------
# Add Surfaces
feb.add_surfaces({"ENDO": np.vstack(lv.get_surface_oi(LV_SURFS.ENDO)) + 1})

# ----------------------
# Add Discrete set
feb.add_discretesets(
  {"AORTIC_RIM_ELEMS": adjusted_discrete_sets["AORTIC_BC"],
   "MITRAL_RIM_ELEMS": adjusted_discrete_sets["MITRAL_BC"]}
  )

# modify existing discrete_set name from template
discrete = feb.discrete()
discrete.find("discrete").attrib["discrete_set"] = "AORTIC_RIM_ELEMS"

# add new 'discrete element to template
import xml.etree.ElementTree as ET
subel = ET.SubElement(discrete, "discrete")
subel.set("discrete_set","MITRAL_RIM_ELEMS")
subel.set("dmat","1")

# ----------------------
# Add fibers
feb.add_meshdata([
  {
    "elem_set": "LV", 
    "var": "mat_axis",
    "elems": {
      "a": lv.get(GEO_DATA.MESH_CELL_DATA, LV_FIBERS.F0),
      "d": lv.get(GEO_DATA.MESH_CELL_DATA, LV_FIBERS.S0),
      }
    }
  ])

# ----------------------
# Check modified feb object
feb

FEBio_feb:
-> Module: 0
-> Control: 19
-> Material: 2
--> material 'Holzapfel_Ogden_PAQ': 13
--> material 'rigid body': 2
-> Globals: 1
-> Geometry: 24
--> Nodes 'LV': 64452
--> Nodes 'AORTIC_RIM': 128
--> Nodes 'MITRAL_RIM': 128
--> Elements 'LV': 218359
--> Elements 'AORTIC_RIM_ELEMS': 63
--> Elements 'MITRAL_RIM_ELEMS': 63
-> Boundary: 1
-> Loads: 1
-> Discrete: 3
-> LoadData: 3
-> Output: 1
-> MeshData: 1

In [12]:
# ----------------------
# Save
feb.write("../sample_files/sample_lv_template_mod.feb")